In [3]:
import keras
from keras.layers import Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D
from keras.models import Sequential
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.utils import np_utils
from sklearn import metrics
from sklearn.metrics import classification_report
import librosa
import librosa.display
import numpy as np
import pandas as pd
import random
import warnings
import os
import glob
import pickle
import random

In [3]:
def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name, sr=16000, mono=True, res_type='kaiser_fast')
    X = librosa.util.normalize(X)
    #IS THE NORMALIZATION ABOVE CORRECT? OR NORMALIZE MFCCS INSTEAD OF FILE
    mfccs = librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=14)
    return mfccs

#iterates over all the files within subdirectories and calls extract_feature
def parse_audio_files(parent_dir,sub_dirs,file_ext='*.wav'):
    features = []
    labels = []
    for label, sub_dir in enumerate(sub_dirs):
        for fn in glob.glob(os.path.join(parent_dir, sub_dir, file_ext)):
            mfccs = extract_feature(fn)
            if mfccs.shape == (14, 63): #IF I REVERT TO OLD PREPROCESS, HANGE ALL 63 TO 87
                features.append(mfccs)
                labels.append(fn.split('/')[2].split('-')[1])
    new_labels = [] #we need the labels to be numbers, not letters.
    for i in labels:
        if i == 'w':
            new_labels.append(0)
        if i == 't':
            new_labels.append(1)
        if i == 's':
            new_labels.append(2)
    return np.array(features), np.array(new_labels, dtype = np.int)

parent_dir = 'mdataset'
train_sub_dirs = ['finaltrain']
test_sub_dirs = ['finaltest']

train_features, train_labels = parse_audio_files(parent_dir,train_sub_dirs)
test_features, test_labels = parse_audio_files(parent_dir,test_sub_dirs)

#Zip labels and mfccs together to randomize the order
train_data = zip(train_features, train_labels)
test_data = zip(test_features, test_labels)
random.shuffle(train_data)
random.shuffle(test_data)
X_train, y_train = zip(*train_data)
X_test, y_test = zip(*test_data)

#Reshape for encoding.
X_train = np.array([x.reshape( (14, 63, 1) ) for x in X_train])
X_test = np.array([x.reshape( (14, 63, 1) ) for x in X_test])

#One-Hot encoding for classes
y_train = np.array(keras.utils.to_categorical(y_train, 3))
y_test = np.array(keras.utils.to_categorical(y_test, 3))

#Save MFCCS to avoid recomputing them (costly)
np.save('X_train.npy', X_train)
np.save('X_test.npy', X_test)
np.save('y_train.npy', y_train)
np.save('y_test.npy', y_test)
print("data exported.")

data exported.


In [5]:
X_train = np.load('X_train.npy')
X_test = np.load('X_test.npy')
y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')

print(X_train.shape)
print(X_test.shape)

(1673, 14, 45, 1)
(697, 14, 45, 1)
[[0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]


In [138]:
#model = Sequential()

#model.add(Dense(14, input_shape=(14, 63, 1)))
#model.add(Activation('sigmoid'))
#model.add(Dropout(0.05)) #prevent overfitting

#model.add(Dense(14, input_shape=(14, 63, 1)))
#model.add(Activation('relu'))
#model.add(Dropout(0.1))

#model.add(Dense(14, input_shape=(14, 63, 1)))
#model.add(Activation('relu'))
#model.add(Dropout(0.1))

#model.add(Flatten())

#model.add(Dense(3, activation='softmax'))

#model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [77]:
model = Sequential()

model.add(Dense(14, input_shape=(14, 63, 1)))
model.add(Activation('sigmoid'))
#model.add(Dropout(0.05)) #prevent overfitting

model.add(Dense(14))
model.add(Activation('relu'))

model.add(Dense(20))
model.add(Activation('relu'))
#model.add(Dropout(0.1))

model.add(Flatten())

model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [78]:
model.fit(X_train, y_train, batch_size=32, epochs=2)
score, acc = model.evaluate(X_test, y_test, batch_size=32)
print('Test score:', score)
print('Test accuracy:', acc)

Y_test = np.argmax(y_test, axis=1) # Convert one-hot to index
y_pred = model.predict_classes(X_test)
print(classification_report(Y_test, y_pred))

Epoch 1/2
2175/2175 [==============================] - 3s 1ms/step - loss: 0.5908 - acc: 0.7605
Epoch 2/2
905/905 [==============================] - 1s 927us/step
('Test score:', 0.4992281621332327)
('Test accuracy:', 0.8121546963301811)
             precision    recall  f1-score   support

          0       0.82      0.87      0.84       396
          1       0.75      0.93      0.83       277
          2       0.93      0.58      0.72       232

avg / total       0.83      0.81      0.81       905

